In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [42]:
import os
repo_path = os.path.abspath("../../")

from git import Repo
repo = Repo(repo_path)
branch = repo.active_branch

# Put into a convenience function to keep track of for later use
gitb = lambda : [print("Current branch:",r.active_branch.name) for r in [Repo(repo_path)]][0]

<font size=6>
    Sberbank Russian Housing Market
</font>

<hr style="border: solid rgb(0,0,0) 0.0px; background-color: rgb(0,0,0);height: 2.0px;"/>
<font color='red' size=5>
    The show so far ... 
</font>


**Summary**

1. Load data
2. Remove features
    * features missing more than 10%
    * features with high correlations to the other features
        * used hierarchical correlation dropper
3. Stage 1 transformations:
    * Ignored features: 
        * e.g. 'id' and 'datetime' columns
    * Scale num. features
        * perhaps better to wait until after 
            1. cat. encoding is done with ordinals
            2. imputation for (all) features
    * Cat. features
        * used Ordinal and OneHot-encoders
        * used custom encoder to include NANs
    * Retrieve column names        
    * Recast all features as needed
4. Stage 2 transformations:
    * Ignored features: 
        * e.g. 'id' and 'datetime' columns
    * Impute remaining features
        * Used `SimpleImputer`
        * `KNNImputer` failed
    * Retrieve column names
    * Recast all features as needed

5. Modeling

6. Evaluate errors

7. Feature importance

**TODOs**

0. Priorities
   1. $\checkmark$ ~~Remove the target feature for the pipeline?~~
        * if the pipeline is fit for the independent features, then it should work with them alone;
        * whatever preprocessing is needed for the target feature can be done independently, outside of the pipeline 
   
1. $\checkmark$ ~~Get the columns names of the transformed data after the `Pipeline` / `ColumnTransformer` steps~~
    * _Needed for checking, further exploration, and featuring engineering_
    
2. Check on the appropriate strategies
    * $\checkmark$ ~~first do cat. encoding with `ColumnTransformer`, then do scaling on numericals in another `Pipeline`~~
    * ~~do everything in a single `Pipeline` taking care of the steps~~ $\to$ everything done with `ColumnTransformer` instead
        * allows individual selection of features for transformations
    * ~~Drop features with many unfilled values (can't do _after_ imputing) or just impute?, then pipeline~~ 
        * $\checkmark$ ~~Drop features with many unfilled values before/after preprocessing?~~
        * $\checkmark$ ~~impute afterwards~~

3. $\checkmark$ ~~Drop features which are mutually correlated; keep only the most relevant ones~~
    * **!!!** do this before first preprocessing stage?

4. More EDA
    * Identify quantities to be removed:
        * $\checkmark$ ~~IDs which contribute to the spikes in the neg. tail of the 'price_doc' distribution~~ $\to$ _this led to a big step in improvement_
            * ~~or see if they're correlated with another feature, like 'sub_area'~~ $\to$ did not find this; just removed sampled IDs
            * **open** 
                * is it possible to automatically detect narrow-bin spikes in a distribution? and eleminate the rows which contribute to them?
        * $\checkmark$ ~~check the distribution of the 'sub_area' feature~~
            * ~~are there enough samples for each category?~~ 

5. More feature engineering
    * There are a few more things which can be done to see if there's an improvement:
        * $\checkmark$ ~~removing some IDs (rows) which have some outlier behavior (clearly defined spikes in distributions)~~
        * aggregating some features together (e.g. combining nearby 'sub_areas') $\to$ needs some ad hoc investigations
        * $\checkmark$ Using the classifiers to identify and reduce the number of features to only the important ones. $\to$ _no improvement; slightly worse with XGB_
            * $\checkmark$ removed the 'sub_area' feature entirely $\to$ _no improvement_
            * **TODO** try with other classifiers

6. Try to package up the routine into a single stage with a multistep pipeline
    * The issue is employing the KNN_imputer
        * the issue _was_ keeping track of the columns since the stages work on different subsets;
    * An idea would be to
        1. Order the columns first by datatype
        2. Then do the structure-preserving transformations (transformations which preserve the number of columns) like scaling, ordinal encoding, imputing(?))
        3. Then do the transformations like OneHotEnc 
    * **Notes**
        * since ther original plan was to use the KNN-imputer, where all features were needed together, this meant
            1. process the num. and cat. features separately
            2. re-attach columns and re-cast dtypes
            3. apply the KNN-imputer
    * **IDEA** to generalize, need a flexible pipeline 
        * `overall_pipeline([sub_pipeline1(data_subsetA, data_subsetB, data_subsetC), sub_pipeline2(data_subsetA, data_subset_rest),...])`
        * the problem is keeping track of the columns since after the first subpipeline, they will be lost.
            
**Notes**
* All transforms were included with `ColumnTransformer`
    * selects cols individually
    * can act as a pipeline
    * retrieve the columns names and ordering
        * _this is important to do the next steps of processing_

<hr style="border: solid rgb(0,0,0) 0.0px; background-color: rgb(0,0,0);height: 2.0px;"/>

In [43]:
# This will reload imports before executing code, allowing you to easily change contents of custom scripts
%load_ext autoreload
%autoreload 2

In [44]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [45]:
import os

In [46]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import random
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, 20)]
random.shuffle(colors)

In [47]:
#import importlib
import sys
sys.path.append('./helpers/')

## Load the data

In [48]:
df = pd.read_csv('../../datasets/sberbank-russian-housing-market/train.csv', \
                 infer_datetime_format=True, parse_dates=['timestamp'])

In [49]:
df.shape

(30471, 292)

Get the dtypes of the DF; save to a dict for later use

In [50]:
df_dtypes = df.dtypes.reset_index().rename(columns={'index':'column', 0:'datatype'})
df_dtype_dict = dict(zip(df_dtypes.column, df_dtypes.datatype.astype(str)))

## Preprocess the data

### Feature work: the target

#### Separate the target feature

**Note** the target feature may also need to be transformed, _e.g._ rescaled

In [51]:
target_feature = 'price_doc'
target_col = [target_feature]

In [52]:
df[target_col].head()

,price_doc
0,5850000
1,6000000
2,5700000
3,13100000
4,16331452


In [53]:
df_targ = df[['id']+target_col].copy()

In [54]:
df = df.drop(columns=target_feature)

In [55]:
#feature_importance_dropped_cols = ['incineration_raion', 'big_market_raion', 'sub_area']
#df = df.drop(columns=feature_importance_dropped_cols)

#### Inspect and clean the target

In [56]:
df_targ['price_doc'].describe()

count    3.047100e+04
mean     7.123035e+06
std      4.780111e+06
min      1.000000e+05
25%      4.740002e+06
50%      6.274411e+06
75%      8.300000e+06
max      1.111111e+08
Name: price_doc, dtype: float64

In [57]:
targ_vals = df_targ['price_doc'].values

#### filter the smoothed IDs

In [58]:
# Ideally, these should all be chosen from inspection; this was done already in 
# * sberbank_russian_housing__dropping_rows_via_distribution_outliers_exploration.ipynb
# * 
bin_width = 1e5
bins=np.arange(bin_width, 1.2e8 + bin_width, bin_width)
peaks = np.arange(1,20.5,0.5)*1e6 

In [62]:
from distribution_tools import get_smooth_ids

In [63]:
id_list = get_smooth_ids(df_targ, 'id', 'price_doc', peaks, bin_width, False) 

In [64]:
id_list = list(map(int, id_list))
df_targ = df_targ[df_targ['id'].isin(id_list)].copy()
df = df[df['id'].isin(id_list)].copy()

### Feature work: the rest

#### Separate some features based on type and/or use

##### Identify the numerical features

In [65]:
ignore_cols = ['id','timestamp']

In [66]:
float_cols = df.drop(columns=ignore_cols).select_dtypes('float').columns.tolist()

int_cols = df.drop(columns=ignore_cols).select_dtypes('int').columns.tolist()

num_cols = df.drop(columns=ignore_cols).select_dtypes(['int', 'float']).columns.tolist()

Select the categorical features

In [67]:
string_cols = df.drop(columns=ignore_cols).select_dtypes('object').columns.tolist()

### Drop features

#### Characterize the missing values

In [68]:
from helpers.fill_rates import get_column_fill_rates, get_row_fill_rates

In [69]:
df_fillrates = get_column_fill_rates(df,0.0)
unfilled_cols = df_fillrates[df_fillrates['frac_full']<=0.90]['column_name'].tolist()

##### Check the fill rate of the features

Keep all features having more than 90% (arbitrary) of their values

In [70]:
threshold = 0.90
df_fillrates = get_column_fill_rates(df,threshold)
display(df_fillrates.sort_values("frac_full", ascending=True)[:10])

,column_name,frac_full
229,cafe_sum_2000_min_price_avg,0.939471
230,cafe_sum_2000_max_price_avg,0.939471
231,cafe_avg_price_2000,0.939471
252,cafe_sum_3000_min_price_avg,0.965377
254,cafe_avg_price_3000,0.965377
253,cafe_sum_3000_max_price_avg,0.965377
275,cafe_sum_5000_min_price_avg,0.989865
276,cafe_sum_5000_max_price_avg,0.989865
277,cafe_avg_price_5000,0.989865
269,prom_part_5000,0.993763


In [71]:
filled_cols = df_fillrates.column_name.tolist()
print(f'Number of features more than {round(100*threshold)}% filled: { len(filled_cols)}')

Number of features more than 90% filled: 256


##### Check the fill rate of the rows

**Note** this is passive; nothing is done with this info; it will also change when the unfilled features are removed

<font color='red' size=2>
    If this were applied, then the target_feature would need to be included (or matched by keeping the `id`)
</font>

In [72]:
threshold = 0.85
df_fillrates = get_row_fill_rates(df,threshold)
display(df_fillrates.sort_values("frac_full", ascending=True)[:10])

,id,row_counts,frac_full
8441,9098,250,0.859107
26297,28430,250,0.859107
3235,3624,250,0.859107
3281,3670,250,0.859107
8078,8710,250,0.859107
6611,7172,250,0.859107
3380,3773,250,0.859107
8427,9083,250,0.859107
26186,28311,250,0.859107
27202,29388,250,0.859107


In [73]:
print(f'Number of features more than {round(100*threshold)}% filled: { df_fillrates.shape[0]}')

Number of features more than 85% filled: 28217


##### Check the information content of the features

If the features contain little useful information (_ie_ they have low variance), they might not contribute much to the model. 
<br/>
For efficiecy reasons, it might be best to drop these, though the resulting model will likely be unchanged.

In [74]:
num_rows = len(df.index)
low_information_cols = [] #

for col in df.columns:
    cnts = df[col].value_counts(dropna=False)
    top_pct = (cnts/num_rows).iloc[0]
    
    if top_pct > 0.95:
        low_information_cols.append(col)
        print('{0}: {1:.5f}%'.format(col, top_pct*100))
        print(cnts)
        print()

oil_chemistry_raion: 99.04316%
no     27948
yes      270
Name: oil_chemistry_raion, dtype: int64

railroad_terminal_raion: 96.22936%
no     27154
yes     1064
Name: railroad_terminal_raion, dtype: int64

nuclear_reactor_raion: 97.32086%
no     27462
yes      756
Name: nuclear_reactor_raion, dtype: int64

big_road1_1line: 97.46261%
no     27502
yes      716
Name: big_road1_1line, dtype: int64

railroad_1line: 97.04444%
no     27384
yes      834
Name: railroad_1line, dtype: int64

cafe_count_500_price_high: 97.18974%
0    27425
1      754
2       31
3        8
Name: cafe_count_500_price_high, dtype: int64

mosque_count_500: 99.53930%
0    28088
1      130
Name: mosque_count_500, dtype: int64

leisure_count_500: 95.07052%
0    26827
1     1154
2      119
4       48
3       44
9        8
5        7
7        6
6        4
8        1
Name: leisure_count_500, dtype: int64

cafe_count_1000_price_high: 95.56312%
0    26966
1     1027
2      130
3       42
4       33
5       11
6        8
7      

In [75]:
len(low_information_cols)

11

##### Dropping rows of cat. features with low representation

Consider dropping rows which contribute to the tail-end of features' distributions

(_Ideally, they would be outliers in several features, but this is difficult to check with no promise of a significant reward_)

In [76]:
num_rows = len(df.index)
low_rep_cat_cols = [] #

col = 'sub_area'
cnts = df[col].value_counts(dropna=False)
top_pct = (cnts/num_rows).iloc[0]

low_rep_cat_cols.append(col)
print('{0}: {1:.5f}%'.format(col, top_pct*100))
df_cnts = cnts.reset_index().rename(columns={'index':'sub_area', 'sub_area':'counts'})

sub_area: 6.26905%


In [77]:
df_cnts['%'] = df_cnts.counts.apply(lambda x: round(100.*x/num_rows,2))
df_cnts = df_cnts.sort_values('counts',ascending=False)

In [78]:
df_cnts.head(20)

,sub_area,counts,%
0,Poselenie Sosenskoe,1769,6.27
1,Nekrasovka,1600,5.67
2,Poselenie Vnukovskoe,1372,4.86
3,Poselenie Moskovskij,922,3.27
4,Poselenie Voskresenskoe,712,2.52
5,Tverskoe,662,2.35
6,Mitino,654,2.32
7,Poselenie Filimonkovskoe,496,1.76
8,Krjukovo,491,1.74
9,Mar'ino,469,1.66


In [80]:
cats_to_exclude = []
counts = 0
for n,r in df_cnts.sort_values('counts').iterrows():

    if counts/num_rows > 0.01:
        print(counts, counts/num_rows)
        break
    counts += r['counts']
    cats_to_exclude.append(r['sub_area'])

315 0.011163087391027005


In [81]:
print(len(cats_to_exclude))
print(cats_to_exclude)

17
['Poselenie Mihajlovo-Jarcevskoe', 'Poselenie Shhapovskoe', 'Poselenie Kievskij', 'Molzhaninovskoe', 'Poselenie Marushkinskoe', 'Vostochnoe', 'Poselenie Voronovskoe', 'Arbat', 'Poselenie Kokoshkino', 'Poselenie Mosrentgen', 'Poselenie Krasnopahorskoe', "Krasnosel'skoe", 'Severnoe', 'Poselenie Rogovskoe', 'Poselenie Rjazanovskoe', 'Vnukovo', "Zamoskvorech'e"]


In [82]:
df_cnts[df_cnts['sub_area'].isin(cats_to_exclude)]

,sub_area,counts,%
128,Zamoskvorech'e,47,0.17
129,Vnukovo,43,0.15
130,Poselenie Rjazanovskoe,34,0.12
131,Poselenie Rogovskoe,31,0.11
132,Severnoe,31,0.11
133,Krasnosel'skoe,30,0.11
134,Poselenie Krasnopahorskoe,25,0.09
136,Poselenie Mosrentgen,17,0.06
135,Poselenie Kokoshkino,17,0.06
137,Arbat,14,0.05


**Notes** 

Strategies:
* drop the rows with these values $\to$ _means no prediction will be made for these!!!_
* combine these values into a new value, e.g. `'other'`
* (advanced) combine nearby regions of low counts into neighbors with higher counts

<font color='red' size=3>
    Warning: as above, if this were applied, then the target_feature would need to be included (or matched by keeping the `id`)
</font>

#### Correlation-based feature drop

In [83]:
from helpers.correlation_feature_drop import get_dropped_columns

In [84]:
threshold = 0.9

In [85]:
# Note: be sure to include the target feature in the correlation --> the current function requires it.
df_corr = df.drop(columns=ignore_cols+string_cols)
df_corr[target_feature] = df_targ['price_doc']
df_corr = df_corr.corr().abs()

In [86]:
drop_corr_cols = get_dropped_columns(df_corr, target_feature, threshold, False)

Index(['col1', 'col2', 'corr'], dtype='object')
Index(['col1', 'col2', 'corr', 'self', 'price_doc'], dtype='object')


#### Select the cols to keep

In [87]:
keep_cols = [c for c in df.columns.tolist() if (c in filled_cols) & (c not in drop_corr_cols + low_information_cols)]

In [88]:
df = df[keep_cols]
print(df.shape) #(30471, 116)
#df = df[df['sub_area'].isin(cats_to_exclude)==False].copy()
print(df.shape)

(28218, 102)
(28218, 102)


### Transformations

#### Inspect the categorical then do encoding

**Note** need to regrab the new set of string cols since some were dropped.

In [89]:
string_cols = df.drop(columns=ignore_cols).select_dtypes('object').columns.tolist()

In [90]:
print(string_cols)

['product_type', 'sub_area', 'culture_objects_top_25', 'thermal_power_plant_raion', 'incineration_raion', 'radiation_raion', 'big_market_raion', 'detention_facility_raion', 'water_1line', 'ecology']


In [91]:
df[string_cols].head()

,product_type,sub_area,culture_objects_top_25,thermal_power_plant_raion,incineration_raion,radiation_raion,big_market_raion,detention_facility_raion,water_1line,ecology
0,Investment,Bibirevo,no,no,no,no,no,no,no,good
1,Investment,Nagatinskij Zaton,yes,no,no,no,no,no,no,excellent
2,Investment,Tekstil'shhiki,no,no,no,yes,no,no,no,poor
3,Investment,Mitino,no,no,no,no,no,no,no,good
4,Investment,Basmannoe,no,no,no,yes,no,no,no,excellent


**Note**
* Ordinals:
    * 'ecology'
* OneHots
    * 'sub_area', 'product_type' (but only two vals)
* Binaries
    * the rest
    * _But these will also be considered ordinals_

~~<font color='red' size=3>
    EDIT: dropped 'sub_area' functionality
</font>~~

In [92]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer

import categorical_encoding
from categorical_encoding import OrdinalEncoderNans


onehot_cat_cols = ["sub_area", "product_type"]
#onehot_cat_cols = ["product_type"]

ord4_cat_cols = ["ecology"]
ord4_cats = ['poor', 'satisfactory', 'good', 'excellent', 'no data'] # Normally, this is a nested list, ie [[1,2,3,...]]

ord2_cat_cols = [e for e in string_cols if (e not in onehot_cat_cols) & (e not in ord4_cat_cols) ]
ord2_cats = len(ord2_cat_cols)*[["no", "yes"]]

In [93]:
df[onehot_cat_cols].nunique()

sub_area        145
product_type      2
dtype: int64

In [94]:
onehot_sum = sum(df[onehot_cat_cols].nunique())

print(len(num_cols), len(onehot_cat_cols), len(ord2_cat_cols), len(ord4_cat_cols))
sum([len(num_cols), onehot_sum, len(ord2_cat_cols), len(ord4_cat_cols)])

274 2 7 1


429

#### Pipelines

##### Numerical features in a pipeline

**Note** could also use with the ColumnTransformer, like the cat. features, since is only one transform on the subset

In [95]:
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ('std_scaler', StandardScaler()),
    ])
# Use in ColumnTransformer as 
#   ("num_pipe",  num_pipeline, num_cols),


##### putting the cat-encs together into the pipeline

In [96]:
# Need to re-grab the numerics as well!
num_cols = df.drop(columns=ignore_cols).select_dtypes(['int', 'float']).columns.tolist()

In [97]:
pipeline_stage1 = ColumnTransformer([    
        ("ignored",   FunctionTransformer(lambda x: x), ignore_cols),
        #("target",  StandardScaler(), target_col),  
        ("num_cols",  StandardScaler(), num_cols),  
        ("onehots",   OneHotEncoder(sparse=False), onehot_cat_cols),
        ("ordinals2", OrdinalEncoder(ord2_cats), ord2_cat_cols),   
        ("ordinals4", OrdinalEncoderNans(ord4_cats), ord4_cat_cols),    
    ])

data = pipeline_stage1.fit_transform(df)

In [98]:
data.shape

(28218, 247)

In [99]:
df.shape

(28218, 102)

####  Retrieve the column names

This is a bit specific here, but generalizable

In [100]:
def get_transformed_column_names(pipeline_transformers):

    prepro_columns = []
    for line in pipeline_stage1.transformers_[:]:

        #print(type(line[1]).__name__ )
        if type(line[1]).__name__ == 'OneHotEncoder':
            # Note: can include old column identifiers in the OHE feature names
            #    with .get_feature_names(ohe_column_names)
            new_cols = line[1].get_feature_names().tolist()
            #print(line[0],'\t', len(new_cols))
            prepro_columns += new_cols
        else:
            new_cols = line[2]
            #print(line[0],'\t', len(new_cols))
            prepro_columns += new_cols    
            
    return prepro_columns

In [101]:
prepro_columns = get_transformed_column_names(pipeline_stage1.transformers_)

In [102]:
onehot_sum = sum(df[onehot_cat_cols].nunique())
check_col_lengths = [len(ignore_cols), len(target_col), len(num_cols), onehot_sum, len(ord2_cat_cols), len(ord4_cat_cols)]
print(*check_col_lengths)
print(sum(check_col_lengths), '==', len(prepro_columns))

2 1 90 147 7 1
248 == 247


#### Rebuild the DF and re-cast the data type

Prepare for the next pipeline: imputing

##### Rebuild the DF

In [103]:
df = pd.DataFrame(data=data, columns=prepro_columns)

##### Re-cast the data types

In [104]:
ignored_dtypes = [df_dtype_dict[c] for c in ignore_cols]

def set_dtypes(prepro_columns, nonfloat_dtypes):
    """Set a new dtype dict for the preprocessed df
    
    Args:
      prepro_columns ([str]): columns for the dict keys
      nonfloat_dtypes ([str]): datatypes for the non-float datatypes 
                               (assumes these are in the front of prepro_columns)

    Returns:
      dict: dictionary of column to datatype
    """

    nr_float_cols = len(prepro_columns)-len(nonfloat_dtypes)

    #### NOTE: all preprocess cols will now be floats
    dtypes_list = nonfloat_dtypes + (nr_float_cols)*['float']

    return dict(zip(prepro_columns, dtypes_list))    

In [105]:
dtypes_dict = set_dtypes(prepro_columns, ignored_dtypes)

In [106]:
len(dtypes_dict)

247

In [107]:
df = df.astype(dtypes_dict)

In [108]:
df.head()

,id,timestamp,full_sq,floor,area_m,green_zone_part,indust_part,school_education_centers_raion,school_education_centers_top_20_raion,healthcare_centers_raion,...,x1_Investment,x1_OwnerOccupier,culture_objects_top_25,thermal_power_plant_raion,incineration_raion,radiation_raion,big_market_raion,detention_facility_raion,water_1line,ecology
0,1,2011-08-20,-0.285567,-0.695280,-0.559419,-0.183110,-0.997333,0.112404,-0.325408,-0.191712,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1,2,2011-08-23,-0.515732,-0.881820,-0.408937,0.854525,-0.578529,0.976425,-0.325408,-0.191712,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
2,3,2011-08-27,-0.285567,-1.068360,-0.635059,-0.620960,0.003622,0.688418,-0.325408,-0.191712,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,4,2011-09-01,0.890835,0.237419,-0.267325,-0.154878,-0.408564,1.552439,-0.325408,-0.191712,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,5,2011-09-05,0.583948,-0.695280,-0.465260,-1.173188,-0.682630,1.264432,-0.325408,1.824339,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0


## Second stage preprocessing

### Imputing

<font color='green' size=2> 
    Can this be organized with the above into a single pipeline?
    <br/>
    Possibly, since the following stage is an imputer
</font>

In [109]:
from sklearn.impute import SimpleImputer, KNNImputer

**Note** `KNNImputer` failed with a kernel restart. Memory? $\to$ try again

In [110]:
subprepro_columns = [c for c in prepro_columns if c not in ignore_cols + target_col]

In [111]:
pipeline_stage2 = ColumnTransformer([
        ("ignored",   FunctionTransformer(lambda x: x), ignore_cols),
        ('imputer', SimpleImputer(strategy='median'), subprepro_columns),    
    ])
 
####NOTE: still need to Scale the ordinalTransformed feature
data = pipeline_stage2.fit_transform(df)

### Rebuild the DF and re-cast the data type

In [112]:
#### NOTE: can use the same cols and dtypes as before since no further reordering of columns was made
df = pd.DataFrame(data=data, columns=prepro_columns).astype(dtypes_dict)

In [113]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28218 entries, 0 to 28217
Data columns (total 247 columns):
id                                       28218 non-null int64
timestamp                                28218 non-null datetime64[ns]
full_sq                                  28218 non-null float64
floor                                    28218 non-null float64
area_m                                   28218 non-null float64
green_zone_part                          28218 non-null float64
indust_part                              28218 non-null float64
school_education_centers_raion           28218 non-null float64
school_education_centers_top_20_raion    28218 non-null float64
healthcare_centers_raion                 28218 non-null float64
university_top_20_raion                  28218 non-null float64
sport_objects_raion                      28218 non-null float64
additional_education_raion               28218 non-null float64
shopping_centers_raion                   28218 non-nu

## Last transform to the target

In [114]:
stasca = StandardScaler()

In [115]:
df_targ['price_doc_'] = stasca.fit_transform(df_targ['price_doc'].values.reshape(-1, 1))

In [116]:
df_targ.head()

,id,price_doc,price_doc_
0,1,5850000,-0.306309
1,2,6000000,-0.274351
2,3,5700000,-0.338267
3,4,13100000,1.238313
4,5,16331452,1.926777


In [117]:
df_targ.shape

(28218, 3)

In [118]:
df_targ.head()

,id,price_doc,price_doc_
0,1,5850000,-0.306309
1,2,6000000,-0.274351
2,3,5700000,-0.338267
3,4,13100000,1.238313
4,5,16331452,1.926777


In [119]:
df.shape, df_targ.shape

((28218, 247), (28218, 3))

In [120]:
df_out = df.merge(df_targ, on=['id'], how='outer')

<hr style="border: solid rgb(0,0,0) 0.0px; background-color: rgb(0,0,0);height: 4.0px;"/>

# Output

df_out.to_csv('../../datasets/sberbank-russian-housing-market/train_prepro.csv', index=False)

In [121]:
!ls -h -lat ../../datasets/sberbank-russian-housing-market/train_prepro.csv

-rw-r--r-- 1 sandm sandm 66M Mai 28 14:10 ../../datasets/sberbank-russian-housing-market/train_prepro.csv
